# Import Important Libaries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import LearningRateScheduler, LambdaCallback
from tensorflow.keras.optimizers import Adam
from keras.optimizers import RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from datetime import datetime
import os

C:\Users\user\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Seed

In [2]:
seed = 42

In [3]:
data_dir = "imagesTrainGrouped/train"
test_data_dir = "imagesTrainGrouped/test"
log_dir_folder = "tensorboard/tensorboard_logs/"

# Hyperparameter tuning

## Batch Size

In [4]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

    return model

### Batch Size = 32

In [5]:
batch_size = 32

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
    # 80% for training, 20% for validation
)
 
# Load the train and test datasets

 
train_dataset = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed = seed
)
 
validation_dataset = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', 
    seed = seed 
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_dataset = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    seed = seed 
)

Found 495 images belonging to 99 classes.
Found 99 images belonging to 99 classes.
Found 396 images belonging to 99 classes.


#### Validation of batch size = 32

In [7]:
# Print the number of samples in each dataset
print(f"Number of samples in the train dataset: {train_dataset.samples}")
print(f"Number of samples in the validation dataset: {validation_dataset.samples}")
 
# Print the shape of the image
sample_image, _ = train_dataset.next()
print("Shape of the image:", sample_image.shape[1:])
 
# Compile the model
model = create_model()
 
# Set up TensorBoard
log_dir = "tensorboard/tensorboard_logs/" + "ValidBatch32"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
 
# Train the model
model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Number of samples in the train dataset: 495
Number of samples in the validation dataset: 99
Shape of the image: (150, 150, 3)



Epoch 1/30


16/16 [==============================] - 10s 540ms/step - loss: 4.6373 - accuracy: 0.0101 - auc: 0.4923 - val_loss: 4.5898 - val_accuracy: 0.0303 - val_auc: 0.5469
Epoch 2/30
16/16 [==============================] - 9s 580ms/step - loss: 4.5580 - accuracy: 0.0586 - auc: 0.5599 - val_loss: 4.4827 - val_accuracy: 0.0404 - val_auc: 0.5874
Epoch 3/30
16/16 [==============================] - 10s 600ms/step - loss: 4.3327 - accuracy: 0.0808 - auc: 0.6511 - val_loss: 4.1665 - val_accuracy: 0.0707 - val_auc: 0.7046
Epoch 4/30
16/16 [==============================] - 9s 587ms/step - loss: 3.9381 - accuracy: 0.1495 - auc: 0.7360 - val_loss: 4.0145 - val_accuracy: 0.0909 - val_auc: 0.7302
Epoch 5/30
16/16 [==============================] - 9s 588ms/step - loss: 3.5433 - accuracy: 0.2162 - auc: 0.7988 - val_loss: 3.9217 - val_accuracy: 0.2020 - val_auc: 0.73

#### Testing of batch size = 32

In [8]:
test_evaluation = model.evaluate(test_dataset)

# Display the evaluation metrics
print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 125ms/step - loss: 2.6037 - accuracy: 0.5000 - auc: 0.9019
Test Loss: 2.603743076324463
Test Accuracy: 0.5


### Batch Size = 64

In [9]:
batch_size = 64

In [10]:
train_dataset64 = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed = seed
)
 
validation_dataset64 = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', 
    seed = seed 
)


test_dataset64 = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    seed = seed 
)

Found 495 images belonging to 99 classes.
Found 99 images belonging to 99 classes.
Found 396 images belonging to 99 classes.


#### Validation at batch size = 64

In [11]:
log_dir = "tensorboard/tensorboard_logs/" + "Valid-Batch_64"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model = create_model()

# Train the model
model.fit(
    train_dataset64,
    epochs=30,
    validation_data=validation_dataset64,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
8/8 [==============================] - 12s 1s/step - loss: 4.6487 - accuracy: 0.0162 - auc: 0.4974 - val_loss: 4.5727 - val_accuracy: 0.0101 - val_auc: 0.5719
Epoch 2/30
8/8 [==============================] - 10s 1s/step - loss: 4.5343 - accuracy: 0.0465 - auc: 0.5808 - val_loss: 4.4475 - val_accuracy: 0.0404 - val_auc: 0.6091
Epoch 3/30
8/8 [==============================] - 9s 1s/step - loss: 4.3034 - accuracy: 0.0808 - auc: 0.6595 - val_loss: 4.1170 - val_accuracy: 0.0808 - val_auc: 0.7215
Epoch 4/30
8/8 [==============================] - 10s 1s/step - loss: 3.9313 - accuracy: 0.1293 - auc: 0.7495 - val_loss: 3.9196 - val_accuracy: 0.1717 - val_auc: 0.7544
Epoch 5/30
8/8 [==============================] - 10s 1s/step - loss: 3.6627 - accuracy: 0.1818 - auc: 0.7915 - val_loss: 3.7962 - val_accuracy: 0.1313 - val_auc: 0.7831
Epoch 6/30
8/8 [==============================] - 10s 1s/step - loss: 3.3877 - accuracy: 0.2485 - auc: 0.8275 - val_loss: 3.4195 - val_accuracy: 0.2626

#### Testing at batch size = 64

In [12]:
test_evaluation64 = model.evaluate(test_dataset64)

# Display the evaluation metrics
print(f"Test Loss: {test_evaluation64[0]}")
print(f"Test Accuracy: {test_evaluation64[1]}")

7/7 [==============================] - 2s 220ms/step - loss: 4.3110 - accuracy: 0.3333 - auc: 0.8229
Test Loss: 4.3109869956970215
Test Accuracy: 0.3333333432674408


### Batch Size = 128

In [13]:
batch_size = 128

In [14]:
train_dataset128 = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed = seed
)
 
validation_dataset = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', 
    seed = seed 
)


test_dataset128 = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    seed = seed 
)

Found 495 images belonging to 99 classes.
Found 99 images belonging to 99 classes.
Found 396 images belonging to 99 classes.


#### Validation of batch size = 128

In [15]:
log_dir = "tensorboard/tensorboard_logs/" + "Valid-Batch_128"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model = create_model()

# Train the model
model.fit(
    train_dataset128,
    epochs=30,
    validation_data=validation_dataset128,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
4/4 [==============================] - 12s 3s/step - loss: 4.7444 - accuracy: 0.0121 - auc: 0.4732 - val_loss: 4.5841 - val_accuracy: 0.0303 - val_auc: 0.5255
Epoch 2/30
4/4 [==============================] - 10s 3s/step - loss: 4.5843 - accuracy: 0.0121 - auc: 0.5276 - val_loss: 4.5711 - val_accuracy: 0.0101 - val_auc: 0.5510
Epoch 3/30
4/4 [==============================] - 10s 3s/step - loss: 4.5574 - accuracy: 0.0323 - auc: 0.5457 - val_loss: 4.5092 - val_accuracy: 0.0303 - val_auc: 0.5860
Epoch 4/30
4/4 [==============================] - 10s 2s/step - loss: 4.4522 - accuracy: 0.0384 - auc: 0.6125 - val_loss: 4.3588 - val_accuracy: 0.0404 - val_auc: 0.6500
Epoch 5/30
4/4 [==============================] - 9s 2s/step - loss: 4.2735 - accuracy: 0.0606 - auc: 0.6757 - val_loss: 4.2155 - val_accuracy: 0.0707 - val_auc: 0.6948
Epoch 6/30
4/4 [==============================] - 10s 3s/step - loss: 4.0267 - accuracy: 0.0990 - auc: 0.7367 - val_loss: 4.0674 - val_accuracy: 0.0606

#### Testing of batch size = 128

In [16]:
test_evaluation128 = model.evaluate(test_dataset128)

# Display the evaluation metrics
print(f"Test Loss: {test_evaluation128[0]}")
print(f"Test Accuracy: {test_evaluation128[1]}")

4/4 [==============================] - 2s 362ms/step - loss: 2.8430 - accuracy: 0.3813 - auc: 0.8897
Test Loss: 2.843040943145752
Test Accuracy: 0.38131314516067505


## Number of Layers

### Add 1 Conv2D and Max Pooling Layer

#### Validation

In [17]:
add1_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
add1_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Add_1Layer_Model"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
add1_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 12s 680ms/step - loss: 4.6127 - accuracy: 0.0020 - auc: 0.4711 - val_loss: 4.5930 - val_accuracy: 0.0101 - val_auc: 0.5258
Epoch 2/30
16/16 [==============================] - 10s 648ms/step - loss: 4.5913 - accuracy: 0.0121 - auc: 0.4992 - val_loss: 4.5685 - val_accuracy: 0.0101 - val_auc: 0.5163
Epoch 3/30
16/16 [==============================] - 10s 636ms/step - loss: 4.5126 - accuracy: 0.0242 - auc: 0.5628 - val_loss: 4.3752 - val_accuracy: 0.0303 - val_auc: 0.6677
Epoch 4/30
16/16 [==============================] - 10s 634ms/step - loss: 4.2851 - accuracy: 0.0566 - auc: 0.6580 - val_loss: 4.1120 - val_accuracy: 0.0909 - val_auc: 0.7090
Epoch 5/30
16/16 [==============================] - 10s 627ms/step - loss: 3.9988 - accuracy: 0.0929 - auc: 0.7365 - val_loss: 3.8845 - val_accuracy: 0.1212 - val_auc: 0.7543
Epoch 6/30
16/16 [==============================] - 10s 629ms/step - loss: 3.7470 - accuracy: 0.1737 - auc: 0.7711 - val_loss

#### Testing

In [18]:
test_evaluation = add1_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 136ms/step - loss: 2.4358 - accuracy: 0.5101 - auc: 0.9196
Test Loss: 2.4357876777648926
Test Accuracy: 0.5101010203361511


### Add 2 Conv2D and Max Pooling Layer

#### Validation

In [19]:
add2_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
add2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Add_2Layer_Model"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
add2_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 700ms/step - loss: 4.6056 - accuracy: 0.0081 - auc: 0.4553 - val_loss: 4.5948 - val_accuracy: 0.0101 - val_auc: 0.5024
Epoch 2/30
16/16 [==============================] - 11s 686ms/step - loss: 4.5953 - accuracy: 0.0101 - auc: 0.4857 - val_loss: 4.5892 - val_accuracy: 0.0101 - val_auc: 0.5082
Epoch 3/30
16/16 [==============================] - 10s 650ms/step - loss: 4.5585 - accuracy: 0.0141 - auc: 0.5138 - val_loss: 4.4708 - val_accuracy: 0.0303 - val_auc: 0.5740
Epoch 4/30
16/16 [==============================] - 11s 662ms/step - loss: 4.4367 - accuracy: 0.0182 - auc: 0.5971 - val_loss: 4.3477 - val_accuracy: 0.0202 - val_auc: 0.6363
Epoch 5/30
16/16 [==============================] - 11s 687ms/step - loss: 4.2501 - accuracy: 0.0424 - auc: 0.6685 - val_loss: 4.1255 - val_accuracy: 0.0707 - val_auc: 0.7183
Epoch 6/30
16/16 [==============================] - 10s 656ms/step - loss: 4.0396 - accuracy: 0.0889 - auc: 0.7256 - val_loss

#### Testing

In [20]:
test_evaluation = add2_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 145ms/step - loss: 2.5899 - accuracy: 0.4167 - auc: 0.9139
Test Loss: 2.5899293422698975
Test Accuracy: 0.4166666567325592


### Add 2 Conv2D and Max Pooling Layer but last 2 layers have same number of filters

#### Validation

In [21]:
add2_last_same_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
add2_last_same_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Add_2Layer_Model_Last2Layers_same_channels"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
add2_last_same_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 673ms/step - loss: 4.6084 - accuracy: 0.0020 - auc: 0.4690 - val_loss: 4.5928 - val_accuracy: 0.0303 - val_auc: 0.5247
Epoch 2/30
16/16 [==============================] - 10s 634ms/step - loss: 4.5892 - accuracy: 0.0101 - auc: 0.4968 - val_loss: 4.5523 - val_accuracy: 0.0101 - val_auc: 0.5486
Epoch 3/30
16/16 [==============================] - 10s 617ms/step - loss: 4.4784 - accuracy: 0.0162 - auc: 0.5634 - val_loss: 4.3020 - val_accuracy: 0.0404 - val_auc: 0.6317
Epoch 4/30
16/16 [==============================] - 10s 646ms/step - loss: 4.2731 - accuracy: 0.0525 - auc: 0.6560 - val_loss: 4.1862 - val_accuracy: 0.0202 - val_auc: 0.6613
Epoch 5/30
16/16 [==============================] - 10s 646ms/step - loss: 4.0743 - accuracy: 0.0646 - auc: 0.7145 - val_loss: 4.0061 - val_accuracy: 0.0909 - val_auc: 0.7378
Epoch 6/30
16/16 [==============================] - 10s 628ms/step - loss: 3.8587 - accuracy: 0.1172 - auc: 0.7588 - val_loss

#### Testing

In [35]:
test_evaluation = add2_last_same_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 133ms/step - loss: 2.4318 - accuracy: 0.4747 - auc: 0.9193
Test Loss: 2.431769371032715
Test Accuracy: 0.4747474789619446


### Add 2 Conv2D and Max Pooling Layer but last 3 layers have same number of filters

#### Validation

In [23]:
add2_last3_same_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
add2_last3_same_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Added_2Layer_Model_3Layers_same_channels"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
add2_last3_same_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 11s 588ms/step - loss: 4.6045 - accuracy: 0.0040 - auc: 0.4748 - val_loss: 4.5939 - val_accuracy: 0.0202 - val_auc: 0.5062
Epoch 2/30
16/16 [==============================] - 9s 560ms/step - loss: 4.5908 - accuracy: 0.0101 - auc: 0.5012 - val_loss: 4.5450 - val_accuracy: 0.0202 - val_auc: 0.5582
Epoch 3/30
16/16 [==============================] - 9s 589ms/step - loss: 4.4734 - accuracy: 0.0162 - auc: 0.5871 - val_loss: 4.3308 - val_accuracy: 0.0404 - val_auc: 0.6593
Epoch 4/30
16/16 [==============================] - 10s 608ms/step - loss: 4.2504 - accuracy: 0.0586 - auc: 0.6733 - val_loss: 4.0691 - val_accuracy: 0.1010 - val_auc: 0.7286
Epoch 5/30
16/16 [==============================] - 11s 667ms/step - loss: 3.9748 - accuracy: 0.0929 - auc: 0.7583 - val_loss: 3.7832 - val_accuracy: 0.1414 - val_auc: 0.7932
Epoch 6/30
16/16 [==============================] - 9s 573ms/step - loss: 3.7331 - accuracy: 0.1253 - auc: 0.8018 - val_loss: 3

#### Testing

In [36]:
test_evaluation = add2_last3_same_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 122ms/step - loss: 2.2080 - accuracy: 0.4773 - auc: 0.9263
Test Loss: 2.207977294921875
Test Accuracy: 0.47727271914482117


### Base Model

#### Validation

In [25]:
base_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Base_Model"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
base_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 751ms/step - loss: 4.6265 - accuracy: 0.0162 - auc: 0.4831 - val_loss: 4.5829 - val_accuracy: 0.0606 - val_auc: 0.5714
Epoch 2/30
16/16 [==============================] - 10s 595ms/step - loss: 4.5245 - accuracy: 0.0444 - auc: 0.5811 - val_loss: 4.4145 - val_accuracy: 0.0707 - val_auc: 0.6315
Epoch 3/30
16/16 [==============================] - 12s 753ms/step - loss: 4.2015 - accuracy: 0.0727 - auc: 0.6982 - val_loss: 4.0719 - val_accuracy: 0.0909 - val_auc: 0.7336
Epoch 4/30
16/16 [==============================] - 10s 645ms/step - loss: 3.7661 - accuracy: 0.1838 - auc: 0.7677 - val_loss: 3.7415 - val_accuracy: 0.2121 - val_auc: 0.7801
Epoch 5/30
16/16 [==============================] - 11s 709ms/step - loss: 3.4942 - accuracy: 0.2202 - auc: 0.8134 - val_loss: 3.5191 - val_accuracy: 0.2020 - val_auc: 0.8279
Epoch 6/30
16/16 [==============================] - 10s 617ms/step - loss: 3.1953 - accuracy: 0.2586 - auc: 0.8558 - val_loss

#### Testing

In [59]:
test_evaluation = base_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 128ms/step - loss: 2.6791 - accuracy: 0.4672 - auc: 0.8968
Test Loss: 2.6791393756866455
Test Accuracy: 0.46717172861099243


### Remove 1 Conv2D and Max Pooling Layer From Base Model

#### Validation

In [27]:
less_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
less_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder  + "Less_Model"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
less_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 14s 829ms/step - loss: 5.0346 - accuracy: 0.0202 - auc: 0.4883 - val_loss: 4.5945 - val_accuracy: 0.0000e+00 - val_auc: 0.4904
Epoch 2/30
16/16 [==============================] - 14s 785ms/step - loss: 4.5812 - accuracy: 0.0586 - auc: 0.5539 - val_loss: 4.5589 - val_accuracy: 0.0303 - val_auc: 0.5703
Epoch 3/30
16/16 [==============================] - 10s 616ms/step - loss: 4.4695 - accuracy: 0.0586 - auc: 0.6223 - val_loss: 4.3015 - val_accuracy: 0.0909 - val_auc: 0.6883
Epoch 4/30
16/16 [==============================] - 10s 604ms/step - loss: 4.0710 - accuracy: 0.1333 - auc: 0.7229 - val_loss: 4.0714 - val_accuracy: 0.0606 - val_auc: 0.7369
Epoch 5/30
16/16 [==============================] - 10s 620ms/step - loss: 3.6692 - accuracy: 0.2182 - auc: 0.7851 - val_loss: 3.5952 - val_accuracy: 0.1919 - val_auc: 0.8222
Epoch 6/30
16/16 [==============================] - 9s 597ms/step - loss: 3.2915 - accuracy: 0.2465 - auc: 0.8333 - val_l

#### Testing

In [65]:
test_evaluation = less_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 106ms/step - loss: 3.2120 - accuracy: 0.3788 - auc: 0.8716
Test Loss: 3.2120115756988525
Test Accuracy: 0.3787878751754761


### Remove 1 Conv2D and Max Pooling Layer From Base Model but 128 output filters for last Conv Layer

#### Validation

In [29]:
less_modified_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
less_modified_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Less_Modified_Model"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
less_modified_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 29s 2s/step - loss: 5.6673 - accuracy: 0.0121 - auc: 0.5115 - val_loss: 4.5883 - val_accuracy: 0.0202 - val_auc: 0.5392
Epoch 2/30
16/16 [==============================] - 20s 1s/step - loss: 4.5889 - accuracy: 0.0384 - auc: 0.5480 - val_loss: 4.5793 - val_accuracy: 0.0606 - val_auc: 0.6050
Epoch 3/30
16/16 [==============================] - 15s 931ms/step - loss: 4.5359 - accuracy: 0.0687 - auc: 0.5701 - val_loss: 4.4293 - val_accuracy: 0.0606 - val_auc: 0.6453
Epoch 4/30
16/16 [==============================] - 15s 936ms/step - loss: 4.2815 - accuracy: 0.0869 - auc: 0.6644 - val_loss: 4.1349 - val_accuracy: 0.0909 - val_auc: 0.7094
Epoch 5/30
16/16 [==============================] - 14s 909ms/step - loss: 3.8504 - accuracy: 0.1394 - auc: 0.7678 - val_loss: 3.7320 - val_accuracy: 0.1818 - val_auc: 0.7981
Epoch 6/30
16/16 [==============================] - 14s 903ms/step - loss: 3.5624 - accuracy: 0.2081 - auc: 0.8072 - val_loss: 3.62

#### Testing

In [66]:
test_evaluation = less_modified_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 166ms/step - loss: 2.9837 - accuracy: 0.4318 - auc: 0.8818
Test Loss: 2.9836602210998535
Test Accuracy: 0.4318181872367859


### Add 1 Layer Model with 16 Batch Size

In [31]:
batch_size = 16
train_dataset16 = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed = seed
)
 
validation_dataset16 = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', 
    seed = seed 
)


test_dataset16 = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    seed = seed 
)

Found 495 images belonging to 99 classes.
Found 99 images belonging to 99 classes.
Found 396 images belonging to 99 classes.


#### Validation

In [32]:
add1_16BatchSize = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the add1_16BatchSize
add1_16BatchSize.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Add_1Layer_16BatchSize_Model"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
add1_16BatchSize.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 702ms/step - loss: 4.6148 - accuracy: 0.0101 - auc: 0.4715 - val_loss: 4.5929 - val_accuracy: 0.0101 - val_auc: 0.5305
Epoch 2/30
16/16 [==============================] - 11s 695ms/step - loss: 4.5833 - accuracy: 0.0182 - auc: 0.5267 - val_loss: 4.5272 - val_accuracy: 0.0303 - val_auc: 0.5798
Epoch 3/30
16/16 [==============================] - 11s 684ms/step - loss: 4.4579 - accuracy: 0.0364 - auc: 0.5936 - val_loss: 4.3194 - val_accuracy: 0.0404 - val_auc: 0.6739
Epoch 4/30
16/16 [==============================] - 10s 665ms/step - loss: 4.1458 - accuracy: 0.0667 - auc: 0.7011 - val_loss: 4.0058 - val_accuracy: 0.1010 - val_auc: 0.7578
Epoch 5/30
16/16 [==============================] - 11s 701ms/step - loss: 3.8126 - accuracy: 0.1495 - auc: 0.7610 - val_loss: 3.6029 - val_accuracy: 0.1818 - val_auc: 0.8229
Epoch 6/30
16/16 [==============================] - 11s 696ms/step - loss: 3.4483 - accuracy: 0.1838 - auc: 0.8326 - val_loss

#### Testing

In [34]:
test_evalution = add1_16BatchSize.evaluate(test_dataset)

print(f"Test Loss: {test_evalution[0]}")
print(f"Test Accuracy: {test_evalution[1]}")

13/13 [==============================] - 2s 136ms/step - loss: 2.4416 - accuracy: 0.4924 - auc: 0.9153
Test Loss: 2.4416146278381348
Test Accuracy: 0.49242424964904785


## Dropout

### Dropout at 0.25

#### Validation

In [53]:
dropoutModel = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])
dropoutModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])
log_dir = "tensorboard/tensorboard_logs/" + "Batch32_Dropout0.25"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
 
# Train the model
dropoutModel.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 735ms/step - loss: 4.6767 - accuracy: 0.0101 - auc: 0.4834 - val_loss: 4.5898 - val_accuracy: 0.0101 - val_auc: 0.5242
Epoch 2/30
16/16 [==============================] - 11s 689ms/step - loss: 4.5711 - accuracy: 0.0242 - auc: 0.5498 - val_loss: 4.5032 - val_accuracy: 0.0404 - val_auc: 0.5976
Epoch 3/30
16/16 [==============================] - 11s 695ms/step - loss: 4.3174 - accuracy: 0.0626 - auc: 0.6544 - val_loss: 4.1075 - val_accuracy: 0.0909 - val_auc: 0.7018
Epoch 4/30
16/16 [==============================] - 11s 725ms/step - loss: 3.9057 - accuracy: 0.1596 - auc: 0.7382 - val_loss: 3.9427 - val_accuracy: 0.1212 - val_auc: 0.7439
Epoch 5/30
16/16 [==============================] - 12s 778ms/step - loss: 3.5651 - accuracy: 0.2162 - auc: 0.7949 - val_loss: 3.8188 - val_accuracy: 0.2121 - val_auc: 0.7634
Epoch 6/30
16/16 [==============================] - 12s 730ms/step - loss: 3.2867 - accuracy: 0.2707 - auc: 0.8311 - val_loss

#### Testing

In [54]:
test_evaluationDropOut = dropoutModel.evaluate(test_dataset)

# Display the evaluation metrics
print(f"Test Loss: {test_evaluationDropOut[0]}")
print(f"Test Accuracy: {test_evaluationDropOut[1]}")

13/13 [==============================] - 2s 136ms/step - loss: 2.4989 - accuracy: 0.4874 - auc: 0.9147
Test Loss: 2.4988512992858887
Test Accuracy: 0.4873737394809723


### Dropout at 0.5

#### Validation

In [55]:
dropoutModel2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])
dropoutModel2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])
log_dir = "tensorboard/tensorboard_logs/" + "Batch32_Dropout0.5"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
 
# Train the model
dropoutModel2.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 719ms/step - loss: 4.6168 - accuracy: 0.0020 - auc: 0.4776 - val_loss: 4.5853 - val_accuracy: 0.0303 - val_auc: 0.5264
Epoch 2/30
16/16 [==============================] - 11s 686ms/step - loss: 4.5547 - accuracy: 0.0465 - auc: 0.5787 - val_loss: 4.4638 - val_accuracy: 0.0808 - val_auc: 0.6089
Epoch 3/30
16/16 [==============================] - 11s 688ms/step - loss: 4.2810 - accuracy: 0.0788 - auc: 0.6697 - val_loss: 4.0243 - val_accuracy: 0.1515 - val_auc: 0.7388
Epoch 4/30
16/16 [==============================] - 11s 693ms/step - loss: 3.8902 - accuracy: 0.1636 - auc: 0.7559 - val_loss: 3.8762 - val_accuracy: 0.1313 - val_auc: 0.7709
Epoch 5/30
16/16 [==============================] - 11s 692ms/step - loss: 3.6296 - accuracy: 0.1818 - auc: 0.7916 - val_loss: 3.7554 - val_accuracy: 0.1919 - val_auc: 0.7810
Epoch 6/30
16/16 [==============================] - 11s 686ms/step - loss: 3.2926 - accuracy: 0.2323 - auc: 0.8444 - val_loss

#### Testing

In [56]:
test_evaluationDropOut2 = dropoutModel2.evaluate(test_dataset)

# Display the evaluation metrics
print(f"Test Loss: {test_evaluationDropOut2[0]}")
print(f"Test Accuracy: {test_evaluationDropOut2[1]}")

13/13 [==============================] - 2s 137ms/step - loss: 2.4331 - accuracy: 0.5152 - auc: 0.9203
Test Loss: 2.433058977127075
Test Accuracy: 0.5151515007019043


### Dropout at 0.75

#### Validation

In [57]:
dropoutModel3 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.75),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])
dropoutModel3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])
log_dir = "tensorboard/tensorboard_logs/" + "Batch32_Dropout0.75"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
 
# Train the model
dropoutModel3.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 777ms/step - loss: 4.6402 - accuracy: 0.0081 - auc: 0.4957 - val_loss: 4.5852 - val_accuracy: 0.0303 - val_auc: 0.5198
Epoch 2/30
16/16 [==============================] - 12s 743ms/step - loss: 4.5649 - accuracy: 0.0283 - auc: 0.5491 - val_loss: 4.5061 - val_accuracy: 0.0101 - val_auc: 0.6065
Epoch 3/30
16/16 [==============================] - 11s 689ms/step - loss: 4.4110 - accuracy: 0.0586 - auc: 0.6098 - val_loss: 4.1893 - val_accuracy: 0.1010 - val_auc: 0.7122
Epoch 4/30
16/16 [==============================] - 11s 696ms/step - loss: 4.0818 - accuracy: 0.1152 - auc: 0.7224 - val_loss: 3.8622 - val_accuracy: 0.1818 - val_auc: 0.7919
Epoch 5/30
16/16 [==============================] - 12s 728ms/step - loss: 3.7046 - accuracy: 0.1798 - auc: 0.7872 - val_loss: 3.7441 - val_accuracy: 0.1818 - val_auc: 0.7935
Epoch 6/30
16/16 [==============================] - 11s 709ms/step - loss: 3.6530 - accuracy: 0.1818 - auc: 0.8030 - val_loss

#### Testing

In [58]:
test_evaluationDropOut3 = dropoutModel3.evaluate(test_dataset)

# Display the evaluation metrics
print(f"Test Loss: {test_evaluationDropOut3[0]}")
print(f"Test Accuracy: {test_evaluationDropOut3[1]}")

13/13 [==============================] - 2s 141ms/step - loss: 2.3355 - accuracy: 0.4015 - auc: 0.9213
Test Loss: 2.335533380508423
Test Accuracy: 0.4015151560306549


## Optimizer

### SGD

#### Validation

In [43]:
sgd_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
sgd_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "SGD"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
sgd_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 12s 736ms/step - loss: 4.5977 - accuracy: 0.0061 - auc: 0.4943 - val_loss: 4.5964 - val_accuracy: 0.0101 - val_auc: 0.4936
Epoch 2/30
16/16 [==============================] - 11s 703ms/step - loss: 4.5960 - accuracy: 0.0101 - auc: 0.5123 - val_loss: 4.5923 - val_accuracy: 0.0101 - val_auc: 0.5191
Epoch 3/30
16/16 [==============================] - 11s 678ms/step - loss: 4.5960 - accuracy: 0.0101 - auc: 0.5045 - val_loss: 4.5921 - val_accuracy: 0.0101 - val_auc: 0.4931
Epoch 4/30
16/16 [==============================] - 11s 707ms/step - loss: 4.5926 - accuracy: 0.0101 - auc: 0.5063 - val_loss: 4.5927 - val_accuracy: 0.0101 - val_auc: 0.5215
Epoch 5/30
16/16 [==============================] - 11s 679ms/step - loss: 4.5926 - accuracy: 0.0081 - auc: 0.5167 - val_loss: 4.5883 - val_accuracy: 0.0101 - val_auc: 0.5435
Epoch 6/30
16/16 [==============================] - 11s 693ms/step - loss: 4.5894 - accuracy: 0.0121 - auc: 0.5352 - val_loss

#### Testing

In [44]:
test_evaluation = sgd_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 150ms/step - loss: 4.4005 - accuracy: 0.0631 - auc: 0.6548
Test Loss: 4.400512218475342
Test Accuracy: 0.06313131004571915


### Adam

#### Validation

In [45]:
adam_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
adam_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Adam"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
adam_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 756ms/step - loss: 4.6164 - accuracy: 0.0061 - auc: 0.4635 - val_loss: 4.5938 - val_accuracy: 0.0101 - val_auc: 0.5154
Epoch 2/30
16/16 [==============================] - 11s 737ms/step - loss: 4.5893 - accuracy: 0.0121 - auc: 0.5110 - val_loss: 4.5538 - val_accuracy: 0.0303 - val_auc: 0.5594
Epoch 3/30
16/16 [==============================] - 11s 715ms/step - loss: 4.4896 - accuracy: 0.0303 - auc: 0.5851 - val_loss: 4.3337 - val_accuracy: 0.0505 - val_auc: 0.6427
Epoch 4/30
16/16 [==============================] - 11s 712ms/step - loss: 4.2096 - accuracy: 0.0869 - auc: 0.6655 - val_loss: 4.2024 - val_accuracy: 0.1010 - val_auc: 0.6959
Epoch 5/30
16/16 [==============================] - 11s 692ms/step - loss: 3.9572 - accuracy: 0.1232 - auc: 0.7323 - val_loss: 3.7819 - val_accuracy: 0.1717 - val_auc: 0.7743
Epoch 6/30
16/16 [==============================] - 10s 659ms/step - loss: 3.6897 - accuracy: 0.1657 - auc: 0.7912 - val_loss

#### Testing

In [46]:
test_evaluation = adam_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 152ms/step - loss: 2.4714 - accuracy: 0.5051 - auc: 0.9209
Test Loss: 2.4714059829711914
Test Accuracy: 0.5050504803657532


### RMS

#### Validation

In [60]:
rms_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
rms_model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "RMS"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
rms_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 724ms/step - loss: 4.6152 - accuracy: 0.0162 - auc: 0.4651 - val_loss: 4.5925 - val_accuracy: 0.0303 - val_auc: 0.5363
Epoch 2/30
16/16 [==============================] - 11s 680ms/step - loss: 4.5912 - accuracy: 0.0182 - auc: 0.5189 - val_loss: 4.5482 - val_accuracy: 0.0303 - val_auc: 0.5550
Epoch 3/30
16/16 [==============================] - 11s 716ms/step - loss: 4.4942 - accuracy: 0.0242 - auc: 0.5761 - val_loss: 4.3296 - val_accuracy: 0.0404 - val_auc: 0.6704
Epoch 4/30
16/16 [==============================] - 11s 692ms/step - loss: 4.2314 - accuracy: 0.0545 - auc: 0.6859 - val_loss: 4.0374 - val_accuracy: 0.0707 - val_auc: 0.7205
Epoch 5/30
16/16 [==============================] - 11s 703ms/step - loss: 3.9547 - accuracy: 0.1172 - auc: 0.7375 - val_loss: 3.8036 - val_accuracy: 0.1515 - val_auc: 0.7826
Epoch 6/30
16/16 [==============================] - 12s 766ms/step - loss: 3.7473 - accuracy: 0.1495 - auc: 0.7834 - val_loss

#### Testing

In [61]:
test_evaluation = rms_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 147ms/step - loss: 2.3116 - accuracy: 0.5025 - auc: 0.9092
Test Loss: 2.311586856842041
Test Accuracy: 0.502525269985199


## Regularization

### Regularization Factor 0.0001

#### Validation

In [62]:
regular1_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(1e-4)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
regular1_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Regularization1_0.0001"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
regular1_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 14s 800ms/step - loss: 4.6596 - accuracy: 0.0121 - auc: 0.4640 - val_loss: 4.6408 - val_accuracy: 0.0202 - val_auc: 0.5053
Epoch 2/30
16/16 [==============================] - 12s 754ms/step - loss: 4.6284 - accuracy: 0.0323 - auc: 0.5187 - val_loss: 4.5729 - val_accuracy: 0.0505 - val_auc: 0.5478
Epoch 3/30
16/16 [==============================] - 12s 731ms/step - loss: 4.5148 - accuracy: 0.0323 - auc: 0.5965 - val_loss: 4.3702 - val_accuracy: 0.0606 - val_auc: 0.6739
Epoch 4/30
16/16 [==============================] - 12s 739ms/step - loss: 4.1904 - accuracy: 0.0727 - auc: 0.7090 - val_loss: 4.1142 - val_accuracy: 0.0808 - val_auc: 0.7131
Epoch 5/30
16/16 [==============================] - 12s 749ms/step - loss: 3.9212 - accuracy: 0.1293 - auc: 0.7499 - val_loss: 4.0193 - val_accuracy: 0.1313 - val_auc: 0.7436
Epoch 6/30
16/16 [==============================] - 11s 721ms/step - loss: 3.6375 - accuracy: 0.1657 - auc: 0.8086 - val_loss

#### Testing

In [63]:
test_evaluation = regular1_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 3s 245ms/step - loss: 2.4014 - accuracy: 0.5101 - auc: 0.9293
Test Loss: 2.4014058113098145
Test Accuracy: 0.5101010203361511


### Regularization Factor 0.01

#### Validation

In [64]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime
import os
from keras.optimizers import RMSprop
from tensorflow.keras import regularizers

# Assuming you have already loaded or preprocessed your dataset and created train_dataset, validation_dataset

# Model definition with L2 regularization
regular2_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
regular2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Regularization2_0.01"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
regular2_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 746ms/step - loss: 7.8373 - accuracy: 0.0061 - auc: 0.4666 - val_loss: 6.0791 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 2/30
16/16 [==============================] - 12s 728ms/step - loss: 5.5150 - accuracy: 0.0040 - auc: 0.4974 - val_loss: 5.0836 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 3/30
16/16 [==============================] - 13s 813ms/step - loss: 4.9088 - accuracy: 0.0081 - auc: 0.4969 - val_loss: 4.7707 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 4/30
16/16 [==============================] - 12s 750ms/step - loss: 4.7153 - accuracy: 0.0081 - auc: 0.4960 - val_loss: 4.6730 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 5/30
16/16 [==============================] - 12s 785ms/step - loss: 4.6528 - accuracy: 0.0061 - auc: 0.4932 - val_loss: 4.6357 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 6/30
16/16 [==============================] - 12s 789ms/step - loss: 4.6275 - accuracy: 0.0020 - auc: 0.4908 - val_loss

#### Testing

In [67]:
test_evaluation = regular2_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 159ms/step - loss: 4.5953 - accuracy: 0.0101 - auc: 0.5000
Test Loss: 4.595320701599121
Test Accuracy: 0.010101010091602802


### Regularization Factor 0.001

#### Validation

In [68]:
regular3_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
regular3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Regularization3_0.001"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
regular3_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 13s 719ms/step - loss: 5.0324 - accuracy: 0.0101 - auc: 0.4641 - val_loss: 4.8920 - val_accuracy: 0.0101 - val_auc: 0.4991
Epoch 2/30
16/16 [==============================] - 12s 752ms/step - loss: 4.8268 - accuracy: 0.0121 - auc: 0.4982 - val_loss: 4.7631 - val_accuracy: 0.0101 - val_auc: 0.5143
Epoch 3/30
16/16 [==============================] - 11s 705ms/step - loss: 4.7066 - accuracy: 0.0162 - auc: 0.5378 - val_loss: 4.5340 - val_accuracy: 0.0303 - val_auc: 0.5930
Epoch 4/30
16/16 [==============================] - 11s 696ms/step - loss: 4.4292 - accuracy: 0.0505 - auc: 0.6465 - val_loss: 4.2186 - val_accuracy: 0.0707 - val_auc: 0.7245
Epoch 5/30
16/16 [==============================] - 11s 689ms/step - loss: 4.1658 - accuracy: 0.0808 - auc: 0.7242 - val_loss: 4.1928 - val_accuracy: 0.0808 - val_auc: 0.7314
Epoch 6/30
16/16 [==============================] - 11s 700ms/step - loss: 3.9839 - accuracy: 0.1131 - auc: 0.7638 - val_loss

#### Testing

In [69]:
test_evaluation = regular3_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 140ms/step - loss: 2.7366 - accuracy: 0.4419 - auc: 0.9133
Test Loss: 2.736614465713501
Test Accuracy: 0.4419191777706146


### Regularization Factor 0.00001

#### Validation

In [70]:
regular4_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
regular4_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Regularization4_Adam"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
regular4_model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/30
16/16 [==============================] - 14s 720ms/step - loss: 4.6195 - accuracy: 0.0101 - auc: 0.4614 - val_loss: 4.6033 - val_accuracy: 0.0303 - val_auc: 0.5126
Epoch 2/30
16/16 [==============================] - 11s 689ms/step - loss: 4.6004 - accuracy: 0.0121 - auc: 0.4991 - val_loss: 4.5645 - val_accuracy: 0.0404 - val_auc: 0.5680
Epoch 3/30
16/16 [==============================] - 11s 724ms/step - loss: 4.4459 - accuracy: 0.0283 - auc: 0.6063 - val_loss: 4.2046 - val_accuracy: 0.0505 - val_auc: 0.6878
Epoch 4/30
16/16 [==============================] - 12s 753ms/step - loss: 4.1268 - accuracy: 0.0808 - auc: 0.6913 - val_loss: 3.9599 - val_accuracy: 0.1414 - val_auc: 0.7454
Epoch 5/30
16/16 [==============================] - 11s 671ms/step - loss: 3.8128 - accuracy: 0.1434 - auc: 0.7604 - val_loss: 3.8362 - val_accuracy: 0.1616 - val_auc: 0.7753
Epoch 6/30
16/16 [==============================] - 11s 683ms/step - loss: 3.5191 - accuracy: 0.1919 - auc: 0.8127 - val_loss

#### Testing

In [71]:
test_evaluation = regular4_model.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 148ms/step - loss: 2.0510 - accuracy: 0.5404 - auc: 0.9304
Test Loss: 2.0510032176971436
Test Accuracy: 0.5404040217399597


## Learning Rate

In [72]:
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Different learning rates to experiment with
learning_rates = [0.1, 0.01, 0.001,0.0001]

In [76]:
for lr in learning_rates:
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy','AUC'])
     
    # Set up TensorBoard
    log_dir = "tensorboard/tensorboard_logs/" + f"lr_{lr}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
     
    # Train the model
    model.fit(
        train_dataset,
        epochs=30,
        validation_data=validation_dataset,
        callbacks=[LearningRateScheduler(lr_schedule),tensorboard_callback]
    )
    test_evaluation = model.evaluate(test_dataset)

    print(f"Test Loss: {test_evaluation[0]}")
    print(f"Test Accuracy: {test_evaluation[1]}")
    print("+++++++++++++++++++++++++++++++++++=")

Epoch 1/30
16/16 [==============================] - 13s 733ms/step - loss: 65212.6680 - accuracy: 0.0040 - auc: 0.4446 - val_loss: 4.6938 - val_accuracy: 0.0000e+00 - val_auc: 0.4862 - lr: 0.1000
Epoch 2/30
16/16 [==============================] - 11s 694ms/step - loss: 4.6531 - accuracy: 0.0040 - auc: 0.4414 - val_loss: 4.6053 - val_accuracy: 0.0101 - val_auc: 0.5002 - lr: 0.1000
Epoch 3/30
16/16 [==============================] - 11s 683ms/step - loss: 4.6329 - accuracy: 0.0040 - auc: 0.4492 - val_loss: 4.6027 - val_accuracy: 0.0101 - val_auc: 0.5000 - lr: 0.1000
Epoch 4/30
16/16 [==============================] - 10s 644ms/step - loss: 4.6355 - accuracy: 0.0020 - auc: 0.4353 - val_loss: 4.6020 - val_accuracy: 0.0101 - val_auc: 0.5000 - lr: 0.1000
Epoch 5/30
16/16 [==============================] - 11s 688ms/step - loss: 4.6388 - accuracy: 0.0040 - auc: 0.4512 - val_loss: 4.6035 - val_accuracy: 0.0101 - val_auc: 0.5000 - lr: 0.1000
Epoch 6/30
16/16 [==============================] - 

In [75]:
for lr in learning_rates:
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy','AUC'])
     
    # Set up TensorBoard
    log_dir = "tensorboard/tensorboard_logs/" + f"lr_without_decay_{lr}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
     
    # Train the model
    model.fit(
        train_dataset,
        epochs=30,
        validation_data=validation_dataset,
        callbacks=[tensorboard_callback]
    )
    test_evaluation = model.evaluate(test_dataset)

    print(f"Test Loss: {test_evaluation[0]}")
    print(f"Test Accuracy: {test_evaluation[1]}")
    print("+++++++++++++++++++++++++++++++++++=")

Epoch 1/30
16/16 [==============================] - 12s 695ms/step - loss: 129626.7969 - accuracy: 0.0040 - auc: 0.4654 - val_loss: 4.6099 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 2/30
16/16 [==============================] - 11s 707ms/step - loss: 4.6361 - accuracy: 0.0061 - auc: 0.4495 - val_loss: 4.6069 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 3/30
16/16 [==============================] - 13s 820ms/step - loss: 4.6414 - accuracy: 0.0101 - auc: 0.4462 - val_loss: 4.6049 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 4/30
16/16 [==============================] - 12s 721ms/step - loss: 4.6306 - accuracy: 0.0061 - auc: 0.4539 - val_loss: 4.6009 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 5/30
16/16 [==============================] - 12s 767ms/step - loss: 4.6309 - accuracy: 0.0040 - auc: 0.4382 - val_loss: 4.6016 - val_accuracy: 0.0101 - val_auc: 0.5000
Epoch 6/30
16/16 [==============================] - 11s 695ms/step - loss: 4.6406 - accuracy: 0.0040 - auc: 0.4423 - val

# Final Model

## Number of Epochs = 60 and No Checkpoints

In [77]:
final_model_checkpoints = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
final_model_checkpoints.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Final_Model_withcheckpoints"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "final_model_checkpoint.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

class EarlyStoppingAtMinValAcc(tf.keras.callbacks.Callback):
    def __init__(self, patience=15):
        super(EarlyStoppingAtMinValAcc, self).__init__()
        self.patience = patience
        self.best_val_acc = 0
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        current_val_acc = logs.get('val_accuracy')
        if epoch == 0 or current_val_acc > self.best_val_acc:
            self.best_val_acc = current_val_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
                print(f"\nTraining stopped. No improvement in validation accuracy for {self.patience} epochs.")
                self.model.set_weights(self.best_weights)

# Train the model
final_model_checkpoints.fit(
    train_dataset,
    epochs=60,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback]
)

Epoch 1/60
16/16 [==============================] - 16s 827ms/step - loss: 4.6368 - accuracy: 0.0081 - auc: 0.4666 - val_loss: 4.6035 - val_accuracy: 0.0202 - val_auc: 0.5276
Epoch 2/60
16/16 [==============================] - 12s 775ms/step - loss: 4.6064 - accuracy: 0.0121 - auc: 0.5056 - val_loss: 4.5864 - val_accuracy: 0.0202 - val_auc: 0.5351
Epoch 3/60
16/16 [==============================] - 13s 824ms/step - loss: 4.5540 - accuracy: 0.0222 - auc: 0.5659 - val_loss: 4.3993 - val_accuracy: 0.0606 - val_auc: 0.6676
Epoch 4/60
16/16 [==============================] - 11s 708ms/step - loss: 4.3348 - accuracy: 0.0505 - auc: 0.6501 - val_loss: 4.1497 - val_accuracy: 0.1010 - val_auc: 0.7338
Epoch 5/60
16/16 [==============================] - 11s 667ms/step - loss: 4.0108 - accuracy: 0.1131 - auc: 0.7461 - val_loss: 3.9429 - val_accuracy: 0.1010 - val_auc: 0.7542
Epoch 6/60
16/16 [==============================] - 11s 692ms/step - loss: 3.7896 - accuracy: 0.1455 - auc: 0.7782 - val_loss

In [78]:
test_evaluation = final_model_checkpoints.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 3s 227ms/step - loss: 2.2695 - accuracy: 0.5379 - auc: 0.9304
Test Loss: 2.2694690227508545
Test Accuracy: 0.5378788113594055


## Number of Epochs = 60

In [79]:
final_model_early_stop = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
final_model_early_stop.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Final_Model_with_Early_stop"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "final_model_checkpoint.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

class EarlyStoppingAtMinValAcc(tf.keras.callbacks.Callback):
    def __init__(self, patience=15):
        super(EarlyStoppingAtMinValAcc, self).__init__()
        self.patience = patience
        self.best_val_acc = 0
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        current_val_acc = logs.get('val_accuracy')
        if epoch == 0 or current_val_acc > self.best_val_acc:
            self.best_val_acc = current_val_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
                print(f"\nTraining stopped. No improvement in validation accuracy for {self.patience} epochs.")
                self.model.set_weights(self.best_weights)

# Train the model
final_model_early_stop.fit(
    train_dataset,
    epochs=60,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback, model_checkpoint_callback, EarlyStoppingAtMinValAcc()]
)

Epoch 1/60
16/16 [==============================] - ETA: 0s - loss: 4.6305 - accuracy: 0.0081 - auc: 0.4834
Epoch 1: val_accuracy improved from -inf to 0.02020, saving model to final_model_checkpoint.h5


C:\Users\Seif_Eldin_Sameh\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/16 [==============================] - 13s 756ms/step - loss: 4.6305 - accuracy: 0.0081 - auc: 0.4834 - val_loss: 4.6032 - val_accuracy: 0.0202 - val_auc: 0.5157
Epoch 2/60
16/16 [==============================] - ETA: 0s - loss: 4.6026 - accuracy: 0.0121 - auc: 0.5050
Epoch 2: val_accuracy did not improve from 0.02020
16/16 [==============================] - 10s 643ms/step - loss: 4.6026 - accuracy: 0.0121 - auc: 0.5050 - val_loss: 4.5847 - val_accuracy: 0.0202 - val_auc: 0.5410
Epoch 3/60
16/16 [==============================] - ETA: 0s - loss: 4.5057 - accuracy: 0.0202 - auc: 0.5917
Epoch 3: val_accuracy did not improve from 0.02020
16/16 [==============================] - 11s 670ms/step - loss: 4.5057 - accuracy: 0.0202 - auc: 0.5917 - val_loss: 4.3709 - val_accuracy: 0.0202 - val_auc: 0.6555
Epoch 4/60
16/16 [==============================] - ETA: 0s - loss: 4.2828 - accuracy: 0.0465 - auc: 0.6677
Epoch 4: val_accuracy improved from 0.02020 to 0.10101, saving model to final_mode

In [80]:
test_evaluation = final_model_early_stop.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 146ms/step - loss: 2.1744 - accuracy: 0.5884 - auc: 0.9250
Test Loss: 2.1743767261505127
Test Accuracy: 0.5883838534355164


## Number of Epochs = 90

In [81]:
final_model_early_stop_90 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

# Compile the model
final_model_early_stop_90.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Final_Model_with_Early_stop_90"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "final_model_checkpoint_90.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

class EarlyStoppingAtMinValAcc(tf.keras.callbacks.Callback):
    def __init__(self, patience=15):
        super(EarlyStoppingAtMinValAcc, self).__init__()
        self.patience = patience
        self.best_val_acc = 0
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        current_val_acc = logs.get('val_accuracy')
        if epoch == 0 or current_val_acc > self.best_val_acc:
            self.best_val_acc = current_val_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
                print(f"\nTraining stopped. No improvement in validation accuracy for {self.patience} epochs.")
                self.model.set_weights(self.best_weights)

# Train the model
final_model_early_stop_90.fit(
    train_dataset,
    epochs=90,
    validation_data=validation_dataset,
    callbacks=[tensorboard_callback, model_checkpoint_callback, EarlyStoppingAtMinValAcc()]
)

Epoch 1/90
16/16 [==============================] - ETA: 0s - loss: 4.6354 - accuracy: 0.0081 - auc: 0.4630
Epoch 1: val_accuracy improved from -inf to 0.01010, saving model to final_model_checkpoint_90.h5
16/16 [==============================] - 13s 719ms/step - loss: 4.6354 - accuracy: 0.0081 - auc: 0.4630 - val_loss: 4.6049 - val_accuracy: 0.0101 - val_auc: 0.5326
Epoch 2/90
16/16 [==============================] - ETA: 0s - loss: 4.6066 - accuracy: 0.0061 - auc: 0.4931
Epoch 2: val_accuracy did not improve from 0.01010
16/16 [==============================] - 12s 731ms/step - loss: 4.6066 - accuracy: 0.0061 - auc: 0.4931 - val_loss: 4.5995 - val_accuracy: 0.0101 - val_auc: 0.5220
Epoch 3/90
16/16 [==============================] - ETA: 0s - loss: 4.5879 - accuracy: 0.0141 - auc: 0.5209
Epoch 3: val_accuracy improved from 0.01010 to 0.02020, saving model to final_model_checkpoint_90.h5
16/16 [==============================] - 12s 738ms/step - loss: 4.5879 - accuracy: 0.0141 - auc: 0

In [82]:
test_evaluation = final_model_early_stop_90.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 163ms/step - loss: 2.2427 - accuracy: 0.6162 - auc: 0.9261
Test Loss: 2.242678642272949
Test Accuracy: 0.6161616444587708


## Number of Epochs = 90 with Learning rate scheduler

In [83]:
final_model_early_stop_90_learning_scheduler = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(len(train_dataset.class_indices), activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy','AUC'])
# Compile the model
final_model_early_stop_90_learning_scheduler.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Final_Model_with_Early_stop_90_lr"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "final_model_checkpoint_90_lr.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

class EarlyStoppingAtMinValAcc(tf.keras.callbacks.Callback):
    def __init__(self, patience=15):
        super(EarlyStoppingAtMinValAcc, self).__init__()
        self.patience = patience
        self.best_val_acc = 0
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        current_val_acc = logs.get('val_accuracy')
        if epoch == 0 or current_val_acc > self.best_val_acc:
            self.best_val_acc = current_val_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
                print(f"\nTraining stopped. No improvement in validation accuracy for {self.patience} epochs.")
                self.model.set_weights(self.best_weights)

# Train the model
final_model_early_stop_90_learning_scheduler.fit(
    train_dataset,
    epochs=90,
    validation_data=validation_dataset,
    callbacks=[LearningRateScheduler(lr_schedule),tensorboard_callback, model_checkpoint_callback, EarlyStoppingAtMinValAcc()]
)

Epoch 1/90
16/16 [==============================] - ETA: 0s - loss: 4.6376 - accuracy: 0.0061 - auc: 0.4626
Epoch 1: val_accuracy improved from -inf to 0.01010, saving model to final_model_checkpoint_90_lr.h5
16/16 [==============================] - 11s 583ms/step - loss: 4.6376 - accuracy: 0.0061 - auc: 0.4626 - val_loss: 4.6049 - val_accuracy: 0.0101 - val_auc: 0.5069 - lr: 0.0010
Epoch 2/90
16/16 [==============================] - ETA: 0s - loss: 4.6057 - accuracy: 0.0101 - auc: 0.4958
Epoch 2: val_accuracy did not improve from 0.01010
16/16 [==============================] - 9s 575ms/step - loss: 4.6057 - accuracy: 0.0101 - auc: 0.4958 - val_loss: 4.6027 - val_accuracy: 0.0101 - val_auc: 0.5204 - lr: 0.0010
Epoch 3/90
16/16 [==============================] - ETA: 0s - loss: 4.5961 - accuracy: 0.0101 - auc: 0.5177
Epoch 3: val_accuracy improved from 0.01010 to 0.03030, saving model to final_model_checkpoint_90_lr.h5
16/16 [==============================] - 11s 671ms/step - loss: 4.5

In [84]:
test_evaluation = final_model_early_stop_90_learning_scheduler.evaluate(test_dataset)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

13/13 [==============================] - 2s 141ms/step - loss: 2.1964 - accuracy: 0.4924 - auc: 0.9364
Test Loss: 2.1964409351348877
Test Accuracy: 0.49242424964904785


## Number of Epochs = 90 and Batch Size = 16

In [85]:
batch_size = 16
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
    # 80% for training, 20% for validation
)
 
# Load the train and test datasets

 
train_dataset16 = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed = seed
)
 
validation_dataset16 = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', 
    seed = seed 
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_dataset16 = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    seed = seed 
)

Found 495 images belonging to 99 classes.
Found 99 images belonging to 99 classes.
Found 396 images belonging to 99 classes.


In [87]:
final_model_early_stop_90_batch_16 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(len(train_dataset16.class_indices), activation='softmax')
])

# Compile the model
final_model_early_stop_90_batch_16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

# Set up TensorBoard
log_dir = log_dir_folder + "Final_Model_with_Early_stop_90_batch16"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "final_model_checkpoint_90_batch16.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

class EarlyStoppingAtMinValAcc(tf.keras.callbacks.Callback):
    def __init__(self, patience=15):
        super(EarlyStoppingAtMinValAcc, self).__init__()
        self.patience = patience
        self.best_val_acc = 0
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        current_val_acc = logs.get('val_accuracy')
        if epoch == 0 or current_val_acc > self.best_val_acc:
            self.best_val_acc = current_val_acc
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
                print(f"\nTraining stopped. No improvement in validation accuracy for {self.patience} epochs.")
                self.model.set_weights(self.best_weights)

# Train the model
final_model_early_stop_90_batch_16.fit(
    train_dataset16,
    epochs=90,
    validation_data=validation_dataset16,
    callbacks=[tensorboard_callback, model_checkpoint_callback, EarlyStoppingAtMinValAcc()]
)

Epoch 1/90
31/31 [==============================] - ETA: 0s - loss: 4.6270 - accuracy: 0.0040 - auc: 0.4386
Epoch 1: val_accuracy improved from -inf to 0.01010, saving model to final_model_checkpoint_90_batch16.h5
31/31 [==============================] - 14s 406ms/step - loss: 4.6270 - accuracy: 0.0040 - auc: 0.4386 - val_loss: 4.6046 - val_accuracy: 0.0101 - val_auc: 0.5022
Epoch 2/90
31/31 [==============================] - ETA: 0s - loss: 4.6038 - accuracy: 0.0121 - auc: 0.5089
Epoch 2: val_accuracy did not improve from 0.01010
31/31 [==============================] - 13s 433ms/step - loss: 4.6038 - accuracy: 0.0121 - auc: 0.5089 - val_loss: 4.5933 - val_accuracy: 0.0101 - val_auc: 0.5352
Epoch 3/90
31/31 [==============================] - ETA: 0s - loss: 4.5278 - accuracy: 0.0202 - auc: 0.5490
Epoch 3: val_accuracy improved from 0.01010 to 0.05051, saving model to final_model_checkpoint_90_batch16.h5
31/31 [==============================] - 12s 371ms/step - loss: 4.5278 - accuracy:

In [88]:
test_evaluation = final_model_early_stop_90_batch_16.evaluate(test_dataset16)

print(f"Test Loss: {test_evaluation[0]}")
print(f"Test Accuracy: {test_evaluation[1]}")

25/25 [==============================] - 3s 115ms/step - loss: 2.2769 - accuracy: 0.5606 - auc: 0.9116
Test Loss: 2.276899814605713
Test Accuracy: 0.560606062412262
